In [1]:
import sparse
import torch

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

In [2]:
from src.environment.env_place_pair import GraphSeriesEnvPlacePair
from src.models.ppo_policy import CustomPPOPolicy
from src.utils.callback import CustomTensorboardCallback


In [3]:
torch.cuda.is_available()

False

In [4]:
import yaml

# Load YAML configuration
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)


In [5]:
config

{'circuit': {'circuit': None,
  'random_circuits': True,
  'n_slices': 32,
  'n_qubits': 8,
  'gates_per_slice': [1]},
 'action_type': 'S',
 'weights_reward': {'nonlocal': 10},
 'ppo': {'batch_size': 64,
  'learning_rate': 0.0005,
  'n_steps': 4096,
  'gae_lambda': 0.95},
 'policy_kwargs': {'net_arch': [64, 64, 64],
  'features_extractor_kwargs': {'hidden_features': 64,
   'num_heads': 1,
   'hidden_layers': 4,
   'dropout': 0.3,
   'out_features': 16,
   'gnn_name': 'GATv2'}},
 'total_timesteps': 3000000}

In [6]:
# environment

a = sparse.load_npz('data/cuccaroadder_q8.npz')
#circuit_config = {'circuit': a}


def make_env():
    env = GraphSeriesEnvPlacePair(circuit_config=config['circuit'], action_type=config['action_type'], weights_reward=config['weights_reward'])
    env = ActionMasker(env, lambda e: e.env_mask())
    #env = Monitor(env)
    return env


env = DummyVecEnv([make_env])
print(env.envs[0])

<ActionMasker<GraphSeriesEnvPlacePair instance>>


In [7]:
env.envs[0].step(0)

({'old_allocation': array([[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]),
  'new_allocation': array([[0., 1.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]),
  'interactions': array([[0, 0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0]]),
  'lookaheads': array([0.00000000e+00, 1.58691406e-02, 0.00000000e+00, 1.90734863e-06,
         1.52587891e-05, 5.21540642e-07, 1.00000000e+00, 0.00000000e+00]),
  'n_qbits': array([8]),
  'core_capacities': array([4, 2])},
 array([-40., -40.]),
 False,
 False,
 {})

In [8]:
# model

load_model = False
load_path = ''
config['policy_kwargs']['device'] = "cuda" if torch.cuda.is_available() else "cpu"

if load_model:
    model = MaskablePPO.load(load_path, env=env, device="cuda" if torch.cuda.is_available() else "cpu")

else:
    model = MaskablePPO(policy=CustomPPOPolicy,
                policy_kwargs=config['policy_kwargs'],
                env = env,
                device="cuda" if torch.cuda.is_available() else "cpu",
                verbose = 1,
                seed = 42,
                tensorboard_log = 'runs',
                **config['ppo']
            )

print(model.device)


Using cpu device
{}
cpu


In [9]:
# train
model_path = 'models/'
run_id = 'test'

callback = CustomTensorboardCallback(save_path = model_path, verbose=1)    

model.learn(
    total_timesteps=config['total_timesteps'],
    callback=callback,
    reset_num_timesteps = False,
    tb_log_name=run_id
)

Logging to runs\test_0


TypeError: nonzero(): argument 'input' (position 1) must be Tensor, not numpy.ndarray